In [ ]:
import warnings, os
import datetime as dt
import itertools as it
from numpy import nan, inf
import numpy as np
import pandas as pd

warnings.filterwarnings('ignore')
pd.set_option('display.width',       None)
pd.set_option('display.max_columns', None)

# Load

In [ ]:
Raw = pd.read_csv('Storage/Metatrader_WINN.csv', parse_dates=['datetime'])
Raw.head()

In [ ]:
def QUERY_SELECT(Raw, SYMBOLS, TFRAMES):
    return Raw[Raw['symbol'].isin(SYMBOLS) & Raw['tf'].isin(TFRAMES)] .reset_index(drop=1)

Query = QUERY_SELECT(Raw, SYMBOLS=['WIN$N'], TFRAMES=['D1'])
Query.head()

# Data

In [ ]:
def FORMAT_SOURCE(Query):
    Src = pd.DataFrame()
    Src[['A','Z']]                      = Query[['a','z']]
    Src[['Symbol','TF']]                = Query[['symbol','tf']]
    Src['Datetime']                     = Query['datetime']
    Src[['Ticks','Volume']]             = Query[['tick_volume','real_volume']]
    Src[['Open','High','Low','Close']]  = Query[['open','high','low','close']]
    Src[['Price']]                      = Query[['close']]

    Src['Hilo']     = Src['High']  - Src['Low']
    Src['Change']   = Src['Close'] - Src['Open']
    Src['Chg Abs']  = Src['Change'].abs()
    return Src 

Src = FORMAT_SOURCE(Query)
Src.head()

In [ ]:
def CALCULATIONS(Src):
    # ================ Helpers ================ #
    def _int(x):
        try:    return int(x)
        except: return x

    # ================ Main ================ #
    Calc = pd.DataFrame(Src)

    BINS_CHG = [100, 200, 500, 800, 1000, 1200, 1500, 1800, 2000, 2200, 2500, 2800, 3000, 3200, 3500, 3800, 4000, 4200, 4500, 4800, 5000, 5500, 6000, 6500, 7000, 7500, 8000]
    BINS_HL  = [500, 1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000, 5500, 6000, 6500, 7000, 7500, 8000, 8500]

    for b in BINS_CHG:  Calc[f'Chg Abs > {b}'] = (Calc['Chg Abs'] > b) 
    for b in BINS_HL:   Calc[f'Hilo > {b}']    = (Calc['Hilo']    > b) 


    for (A,Z) in [x for x in pd.Series([0, *BINS_CHG, inf]).rolling(2) if (len(x) == 2)]:
        Calc[f'Chg Abs {_int(A)}-{_int(Z)}'] = (A <= Calc['Chg Abs']) & (Calc['Chg Abs'] < Z)

    for (A,Z) in [x for x in pd.Series([0, *BINS_HL, inf]).rolling(2) if (len(x) == 2)]:
        Calc[f'Hilo {_int(A)}-{_int(Z)}'] = (A <= Calc['Hilo']) & (Calc['Hilo'] < Z)

    return Calc

Calc = CALCULATIONS(Src)
Calc.head()

# Stats

In [28]:
Calc.loc[:, 'Chg Abs > 100':  'Chg Abs > 5500'].mean().mul(100).round(1)

Chg Abs > 100     93.3
Chg Abs > 200     87.8
Chg Abs > 500     70.4
Chg Abs > 800     52.2
Chg Abs > 1000    41.9
Chg Abs > 1200    32.5
Chg Abs > 1500    23.5
Chg Abs > 1800    16.2
Chg Abs > 2000    11.9
Chg Abs > 2200     9.6
Chg Abs > 2500     5.9
Chg Abs > 2800     3.8
Chg Abs > 3000     2.9
Chg Abs > 3200     2.1
Chg Abs > 3500     1.2
Chg Abs > 3800     0.6
Chg Abs > 4000     0.4
Chg Abs > 4200     0.3
Chg Abs > 4500     0.2
Chg Abs > 4800     0.1
Chg Abs > 5000     0.1
Chg Abs > 5500     0.0
dtype: float64

In [27]:
Calc.loc[:,    'Hilo > 500':     'Hilo > 7500'].mean().mul(100).round(1)

Hilo > 500     100.0
Hilo > 1000     94.9
Hilo > 1500     69.4
Hilo > 2000     39.6
Hilo > 2500     20.7
Hilo > 3000     11.2
Hilo > 3500      4.7
Hilo > 4000      2.5
Hilo > 4500      1.2
Hilo > 5000      0.5
Hilo > 5500      0.2
Hilo > 6000      0.1
Hilo > 6500      0.1
Hilo > 7000      0.1
Hilo > 7500      0.0
dtype: float64

In [26]:
Calc.loc[:, 'Chg Abs 0-100':'Chg Abs 5500-6000'].mean().mul(100).round(1)

Chg Abs 0-100         6.3
Chg Abs 100-200       5.6
Chg Abs 200-500      17.6
Chg Abs 500-800      17.7
Chg Abs 800-1000     10.7
Chg Abs 1000-1200     9.1
Chg Abs 1200-1500     9.6
Chg Abs 1500-1800     6.9
Chg Abs 1800-2000     4.6
Chg Abs 2000-2200     2.4
Chg Abs 2200-2500     3.5
Chg Abs 2500-2800     2.2
Chg Abs 2800-3000     0.8
Chg Abs 3000-3200     0.8
Chg Abs 3200-3500     1.0
Chg Abs 3500-3800     0.5
Chg Abs 3800-4000     0.2
Chg Abs 4000-4200     0.2
Chg Abs 4200-4500     0.2
Chg Abs 4500-4800     0.1
Chg Abs 4800-5000     0.0
Chg Abs 5000-5500     0.1
Chg Abs 5500-6000     0.0
dtype: float64

In [25]:
Calc.loc[:,    'Hilo 0-500':   'Hilo 7500-8000'].mean().mul(100).round(1)

Hilo 0-500         0.0
Hilo 500-1000      4.8
Hilo 1000-1500    25.3
Hilo 1500-2000    30.0
Hilo 2000-2500    19.0
Hilo 2500-3000     9.6
Hilo 3000-3500     6.6
Hilo 3500-4000     2.2
Hilo 4000-4500     1.3
Hilo 4500-5000     0.7
Hilo 5000-5500     0.2
Hilo 5500-6000     0.2
Hilo 6000-6500     0.0
Hilo 6500-7000     0.0
Hilo 7000-7500     0.1
Hilo 7500-8000     0.0
dtype: float64

# Snippets

In [ ]:
def FORMAT_SOURCE(Query):
    Src = pd.DataFrame()
    Src['Hilo']     = Src['High']  - Src['Low']
    Src['Change']   = Src['Close'] - Src['Open']
    Src['Chg Abs']  = Src['Change'].abs()
    return Src 

def CALCULATIONS(Src):
    Calc = pd.DataFrame(Src)
    BINS_CHG = [100, 200, 500, 800, 1000, 1200, 1500, 1800, 2000, 2200, 2500, ...]
    BINS_HL  = [500, 1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000, 5500, ...]

    for b in BINS_CHG:  Calc[f'Chg Abs > {b}'] = (Calc['Chg Abs'] > b) 
    for b in BINS_HL:   Calc[f'Hilo > {b}']    = (Calc['Hilo']    > b) 

    for (A,Z) in [x for x in pd.Series([0, *BINS_CHG, inf]).rolling(2) if (len(x)==2)]:
        Calc[f'Chg Abs {_int(A)}-{_int(Z)}'] = (A <= Calc['Chg Abs']) & (Calc['Chg Abs'] < Z)

    for (A,Z) in [x for x in pd.Series([0, *BINS_HL, inf]).rolling(2) if (len(x)==2)]:
        Calc[f'Hilo {_int(A)}-{_int(Z)}'] = (A <= Calc['Hilo']) & (Calc['Hilo'] < Z)

    return Calc


Calc.loc[:,    'Hilo > 500':     'Hilo > 8500'].mean().mul(100).round(1)
Calc.loc[:, 'Chg Abs > 100':  'Chg Abs > 8000'].mean().mul(100).round(1)
Calc.loc[:,    'Hilo 0-500':   'Hilo 8500-inf'].mean().mul(100).round(1)
Calc.loc[:, 'Chg Abs 0-100':'Chg Abs 8000-inf'].mean().mul(100).round(1)